## Introductie
In deze EDA ga ik de [hunting mantis](https://www.kaggle.com/datasets/stealthtechnologies/predict-hunting-success-rate-of-a-chinese-mantis) dataset onderzoeken, en een machine learning algoritme op trainen.
Uiteindelijk is het bedoeling om te kunnen voorspellen of een jacht van de mantis succesvol is gebasseerd op een aantal variabelen.
Om dit te kunnen doen zal de data eerst onderzocht moeten worden.





## EDA

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### Inlezen

In [2]:
hunting_data = pd.read_csv("./data.csv")
hunting_data.index += 1
hunting_data.head()

,Humidity Level,Temperature,Time of Day,Luring Success,Time Spent Hunting,Prey Size,Ambush,Food Availability,Predator Presence,Is Competition,Hunting Success
1,0.32,31.25,Morning,True,0.81,6.600000,False,NaN,True,False,0
2,0.66,29.08,Afternoon,False,2.81,5.380000,False,Low,False,False,1
3,0.61,25.74,Evening,True,3.79,15.189246,True,Low,False,False,1
4,0.37,22.68,Morning,False,4.27,5.060000,True,Low,True,False,1
5,0.65,29.20,Afternoon,False,3.84,2.710000,False,High,False,False,1


In [17]:
hunting_data.shape

(300000, 11)

Alle kolommen en rijen zijn goed ingelezen, shape komt overeen met wat er op de site staat (300000x11).

In [3]:
codebook = {
    "attribute": hunting_data.columns,
    "unit": ["g/m^3", "Celcius", "NA", "True/False", "Minutes", "Cm", "True/False", "NA", "True/False", "True/False", "1/0"],
    "dtype": ["float64", "float64", "category", "bool", "float64", "float64", "bool", "category", "bool", "bool", "int64"],
    "description": [
        "Humidty level of enviroment",
        "Temperature of enviroment",
        "Time of the day",
        "Luring succes",
        "Time spent hunting",
        "Size of prey being hunted",
        "If mantis used an ambush",
        "Amount of food available",
        "If there are predators near",
        "If there is competition",
        "If the hunt was a succes"
    ]
}
pd.DataFrame(codebook)


,attribute,unit,dtype,description
0,Humidity Level,g/m^3,float64,Humidty level of enviroment
1,Temperature,Celcius,float64,Temperature of enviroment
2,Time of Day,NA,category,Time of the day
3,Luring Success,True/False,bool,Luring succes
4,Time Spent Hunting,Minutes,float64,Time spent hunting
5,Prey Size,Cm,float64,Size of prey being hunted
6,Ambush,True/False,bool,If mantis used an ambush
7,Food Availability,NA,category,Amount of food available
8,Predator Presence,True/False,bool,If there are predators near
9,Is Competition,True/False,bool,If there is competition


Deze dataframe bevat informatie over de attributen van de data

### Univariaat

In [4]:
pd.DataFrame({
  "isna": hunting_data.isna().sum()
})

,isna
Humidity Level,0
Temperature,0
Time of Day,0
Luring Success,0
Time Spent Hunting,0
Prey Size,0
Ambush,0
Food Availability,30212
Predator Presence,29708
Is Competition,30119


Er zijn een aantal missende waarden bij "Food Availability", "Predator Presence", "Is Competition". Ik wil later eerst bepalen hoe relevant deze waarden zijn voordat ik iets aan deze waarden ga doen.

In [5]:
def by_unit(*units):
    return [attribute for attribute, unit in zip(codebook["attribute"], codebook["unit"]) if unit in units]


Om de distributies van numerieke attributen te visualiseren worden boxplotten en histogrammen gebruikt.

In [13]:
hunting_data[by_unit("g/m^3")].boxplot(vert=False, grid=False)

<Axes: >

In [7]:
hunting_data[by_unit("Celcius")].boxplot(vert=False, grid=False)

<Axes: >

In [14]:
hunting_data[by_unit("Cm", "Minutes")].boxplot(vert=False, grid=False)

AssertionError: 1

In [15]:
hunting_data[by_unit("g/m^3", "Celcius", "Minutes", "Cm")].hist(bins=20, layout=(2,2), figsize=(16,9.6));

Het is zichtbaar dat de data van temperatuur uniform verdeeld is.
Bij vochtigheid is het niet volledig univorm, sommige nummers komen iets vaker voor dan andere.
De laatste 2 zijn vrij univorm verdeeld, tot een bepaalde waarde (=>5 bij time spent hunting, => 10 bij Prey size). Dit wijst erop dat over het algemeen er minder dan 5 minuten gejaagd wordt. Grotere prooien komen minder vaak voor, wat kan wijzen naar een voorkeur van de mantis.

Ik wil nu de distributies vergelijken van de categorische attributen, "Time of Day" en "Food Availability". Dit wordt gedaan met een barchart

In [16]:
sns.barplot(hunting_data["Time of Day"]);

In [ ]:
sns.barplot(hunting_data["Food Availability"]);

KeyboardInterrupt: 